A.S. Lundervold, v. 140921

This notebook is partly based on `https://github.com/alu042/DAT158ML/blob/master/notebooks/DAT158-Part1-3-classification.ipynb`, which is partly based on the book _Aurélien Géron &ndash; Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, 2nd Edition_

# Introduction

This notebook continues the story in `2.0-classification-evaluating_binary_classifiers.ipynb` by looking at the evaluation of **multiclass classifiers** through an example. 

# Setup

In [ ]:
# This is a quick check of whether the notebook is currently running on Google Colaboratory, as that makes some difference for the code below.
# We'll do this in every notebook of the course.
if 'google.colab' in str(get_ipython()):
    print('The notebook is running on Colab. colab=True.')
    colab=True
else:
    print('The notebook is not running on Colab. colab=False.')
    colab=False

In [ ]:
# To display plots directly in the notebook:
%matplotlib inline

We import our standard framework:

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib
import sklearn

In [ ]:
# Set the directory in which to store data
NB_DIR = Path.cwd()       # Set NB_DIR to be the current working directory
DATA = NB_DIR/'data'      # The data dir is the subdirectory 'data' under NB_DIR

DATA.mkdir(exist_ok=True) # Create the data directory

# The data: The MNIST data set

We'll base our discussion on a famous benchmark dataset: **MNIST**

In [ ]:
import IPython
IPython.display.Image('assets/MnistExamples.png', width='90%')

MNIST consists of 70.000 examples of handwritten digits. It has been called "the machine learning equivalent of fruit flies": it's simple, but not too simple, and is very well-studied. Have a look at https://en.wikipedia.org/wiki/MNIST_database and http://yann.lecun.com/exdb/mnist/ for more details.

Our goal is to construct a system that can take an image from MNIST as input and produce the correct digit 0, ..., 9 as output. This is a **multi-class system** as there are more than two possible outcomes (in contrast to our previous example of True or False for diabetes).

<centering>
<img src="assets/MNIST-goal.png">
</centering>

Correctly classifying handwritten digits is a difficult problem.. Can you come up with features that characterizes all the number 2's, but none of the other digits? How can you program rules that detect only 2's?

In [ ]:
import IPython
IPython.display.Image('assets/mnist-difficult.png', width='90%')

<span style="font-size:smaller">Image from G. Hinton's Coursera course [Neural Networks for Machine Learning](https://www.coursera.org/learn/neural-networks)</span>

# Get the data

[OpenML](https://www.openml.org/) is a convenient source of machine learning data, containing MNIST among many other standard data sets. Scikit-learn has a method we can use to fetch data from OpenML:

In [ ]:
from sklearn.datasets import fetch_openml

In [ ]:
mnist = fetch_openml('mnist_784', version=1, data_home='./data')

This gives us a Python dictionary containing the data, labels and a description of the data set:

In [ ]:
mnist.keys()

In [ ]:
#print(mnist.DESCR)

Each of the 70.000 images are of size 28*28 = 784, stored as Pandas data frames:

In [ ]:
mnist.data.shape

In [ ]:
mnist.data.head()

There are 70.000 labels, one for each image (i.e. each row of the data frame):

In [ ]:
mnist.target.shape

We store the features in X and the target labels in y, as usual:

In [ ]:
X = mnist.data
y = mnist.target

# Creating training and test sets

As we've discussed, after collecting the data to be studied, the first step is to set aside a test set. 

> We're not interested in how well our models perform on the training set, what we're really after is how well they generalize to unseen data. 

<img width=50% src="assets/testsplit.png"> 

We'll use the first 60.000 images as our training data and the last 10.000 as test (this is the standard split for MNIST):

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

# Explore the data

As always, we should take a look at the training data. Having a feeling for the variation and quality of the data, and also a feeling for the difficulty of the task, is crucial for constructing machine learning models. When dealing with images it's of course convenient to plot them.

In [ ]:
some_digit = np.array(X_train.iloc[34500])

The images are vectors of length 784:

In [ ]:
some_digit.shape

In [ ]:
28*28

To plot them we reshape to 28*28:

In [ ]:
some_digit_image = some_digit.reshape(28,28)

Here's a small section of the image:

In [ ]:
some_digit_image[15:20,15:20]

The numbers represent grayscale values.

In [ ]:
# Plot the image using a grayscale colormap
plt.imshow(some_digit_image, cmap=matplotlib.cm.binary)
plt.axis('off')
plt.show()

Let's make a small convenience function to plot MNIST images:

In [ ]:
def plot_digit(data):
    data = np.array(data)
    image = data.reshape(28,28)
    plt.imshow(image, cmap=matplotlib.cm.binary)
    plt.axis("Off")

In [ ]:
plot_digit(X_train.iloc[1234])

...and plot a random selection:

In [ ]:
import random

In [ ]:
nb=10
to_plot = random.choices(X_train.values, k=nb)

In [ ]:
f = plt.figure(figsize=(14,14))
for i in range(nb):
    plt.subplot(1,nb,i+1)
    plot_digit(to_plot[i])
plt.show()

> **Your turn!** Make a function that plots a random selection of images from a specified class. For example `plot_images(image_class='8', nb=25)` should plot 25 random 8's from the training data:

<img src="assets/plot_MNIST_images.png">

You'll want to create such small throwaway helper functions all the time when you're investigating a new data set.

Here is a solution (I recommend not peeking until you've given it a good shot yourself):

In [ ]:
#%load solutions/2.1-plot_images.py

# Multiclass classification model

In the previous notebook we saw some of the major concepts and techniques in binary classification. Let's try to predict something that's more difficult: all the 10 classes in MNIST.

The ideas behind multiclass predictions are the same, but the difficulty for our models are increased. 

Superficially, not much changes. We can use the same scikit-learn models as before, because scikit-learn takes care of adapting models for multi-class predictions. For example the `SGDClassifier` and the `RandomForestClassifier`.

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
sgd_clf = SGDClassifier(random_state=42)
rf_clf = RandomForestClassifier(random_state=42)

As we've discussed, for the `SGDClassifier` it is important to normalize the data:

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
std = StandardScaler()
X_train_std = std.fit_transform(X_train)
X_test_std = std.transform(X_train)

We can then fit the models and make predictions:

In [ ]:
sgd_clf.fit(X_train_std, y_train)

In [ ]:
# Random forests do not care about normalization, so we can use the original X_train
rf_clf.fit(X_train, y_train)

In [ ]:
some_digit = X_train.iloc[0]
plot_digit(some_digit)

Here are the predictions from our two models:

In [ ]:
sgd_clf.predict([some_digit])

In [ ]:
rf_clf.predict([some_digit])

Behind the scenes, scikit-learn trained 10 binary classifiers for us, and used them all on the `some_digit` data point. The class whose decision score was the highest was the ouput from `predict`:

In [ ]:
sgd_clf.decision_function([some_digit])

In [ ]:
# Return the position of the element with the highest value in the array. 
# That is the model's prediction.

np.argmax(sgd_clf.decision_function([some_digit]))

We can do the same with random forest classifiers by asking for the list of probabilities that the random forest assigned to each class:

In [ ]:
rf_clf.predict_proba([some_digit])

...and notice that the sixth class (corresponding to 5s) got assigned the highest probability.

In [ ]:
np.argmax(rf_clf.predict_proba([some_digit]))

## Evaluation

We can then use the same evaluation techniques that we used in the previous notebook:

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

### Cross-validation

In [ ]:
cross_val_score(sgd_clf, X_train_std, y_train, cv=3, scoring="accuracy", n_jobs=-1)

In [ ]:
cross_val_score(rf_clf, X_train, y_train, cv=3, scoring="accuracy", n_jobs=-1)

### Accuracy on the test set

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test, sgd_clf.predict(X_test))

In [ ]:
accuracy_score(y_test, rf_clf.predict(X_test))

### Confusion matrix


Let's compare the two classifiers' confusion matrices:

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
from utils import plot_confusion_matrix

In [ ]:
sgd_train_pred = cross_val_predict(sgd_clf, X_train_std, y_train, cv=3, n_jobs=-1)
rf_train_pred = cross_val_predict(rf_clf, X_train, y_train, cv=3, n_jobs=-1)

In [ ]:
sgd_cm = confusion_matrix(y_train, sgd_train_pred)
rf_cm = confusion_matrix(y_train, rf_train_pred)

In [ ]:
cms = [sgd_cm, rf_cm]
titles = ["SGDClassifier", "Random Forest"]
fig, axes = plt.subplots(1, 2, figsize=(12,6))

for i, ax in enumerate(axes.flat):
    plot_confusion_matrix(cms[i], classes=sgd_clf.classes_, ax=ax, title=titles[i])

Both confusion matrices look pretty good: most of the images land on the diagonal. However, there are errors and there seems to be some patterns in them... Let's take a closer look.

> As a side note: these are not the kinds of models you would use for computer vision problems these days. You've heard of **"deep learning"** and the **revolution** it has caused in machine learning during the last couple of years. Where it started, and where it's made the most impact until now, is in computer vision. Using deep learning it is _very_ easy to reach models that score more than 99.7% accuracy on MNIST. Which means that the model makes a mistake on *less than 30 images of the 10.000 test images*! And for some of these images, if we take a look ourself, we would agree with the machine's predictions. Modern deep learning has blown away MNIST as a benchmark by essentially "solving" it. 

Another side-note: **Accuracy, precision, recall in multiclass settings**

> In  a previous exercise I asked you to consider how to define precision and recall in a multiclass setting. Think about it again. See if you can figure out how the following numbers are computed: 

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_train, rf_train_pred, digits=3))

# Error analysis

We can zoom in on the errors in the confusion matrices by disregarding the main diagonal. First we have to scale each value by the number of images in the corresponding class, to properly compare error rates:

In [ ]:
row_sums = sgd_cm.sum(axis=1, keepdims=True)
norm_sgd_cm = sgd_cm / row_sums

In [ ]:
_ = plot_confusion_matrix(norm_sgd_cm, classes=sgd_clf.classes_, 
                         title="Error matrix", labels=False)

We notice several interesting things:
- Bright columns for 8 and 9 --> many images get misclassified as 8 or 9s.
- Bright rows for 8 and 9 --> many 8s and 9s are often confused with other digits
- The model confuses for example 7s with 9s and 3s with 5s

This gives us some ideas for improving our model. Perhaps we should try to improve the 8 and 9 classifications, and work to fix the 3/5 confusion.

## Analyzing individual errors

Let's plot some 3s and 5s. Some 3s that the model got correct, some it confused for 5s. Some 5s the confused for 3s, and some correct 5s. 

In [ ]:
# Code taken from the book _Géron: Hands-On Machine Learning with Scikit-Learn and TensorFlow_
def plot_digits(instances, images_per_row=10, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size,size) for instance in instances.values]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap = matplotlib.cm.binary, **options)
    plt.axis("off")

In [ ]:
def individual_errors(clf, cl_a, cl_b):
    cl_a, cl_b = str(cl_a), str(cl_b)
    X_aa = X_train[(y_train == cl_a) & (sgd_train_pred == cl_a)] # Correct class a's
    X_ab = X_train[(y_train == cl_a) & (sgd_train_pred == cl_b)] # a's predicted as b's
    X_ba = X_train[(y_train == cl_b) & (sgd_train_pred == cl_a)] # b's predicted as a's
    X_bb = X_train[(y_train == cl_b) & (sgd_train_pred == cl_b)] # Correct b's
    
    plt.figure(figsize=(10,10))
    plt.subplot(221); plot_digits(X_aa[:25], images_per_row=5)
    plt.subplot(222); plot_digits(X_ab[:25], images_per_row=5)
    plt.subplot(223); plot_digits(X_ba[:25], images_per_row=5)
    plt.subplot(224); plot_digits(X_bb[:25], images_per_row=5)

In [ ]:
individual_errors(sgd_clf, 3, 5)

The SGDClassifier is a linear model. All it does is assign a weight per class to each pixel, and when it sees a new image it just sums up the wighted pixel intensities to get a score for each class. Since 3s and 5s only differ only by a few pixels, this model will easily confuse them. Same for 7s and 9s:

In [ ]:
individual_errors(sgd_clf, 7, 9)

One would expect it to perform better on for example 1s and 4s:

In [ ]:
individual_errors(sgd_clf, 1, 4)

# End

Now you've learned about some of the main practical machine learning ideas and techniques, and you're actually already well-equipped to go out and attack real-world problems.

However, we've treated the models as black boxes. **We haven't really discussed how machine learning models work!** It's important to know some of the how's to be an effective machine learning practitioner.

Next, we'll dig deep into one of the most popular models: the Random Forest.